#Análisis GSEA

El Análisis de Enriquecimiento de Conjuntos de Genes (GSEA) es una herramienta bioinformática utilizada para interpretar conjuntos de genes y sus funciones en experimentos de expresión génica. Su objetivo principal es **determinar si un conjunto específico de genes muestra una distribución significativamente diferente en una lista ordenada de genes en comparación con lo que se esperaría al azar**.

Primero se realiza la preparación de Datos: Se comienza con un conjunto de datos de expresión génica que compara dos o más condiciones experimentales.
Se asocian los genes con información biológica, como vías metabólicas, funciones celulares o procesos biológicos.

A continuación, se ordenan los genes: La lista de genes se ordena según su nivel de expresión diferencial entre las condiciones experimentales. Esto puede basarse en estadísticas de prueba, como el valor p o el cambio en el logaritmo del plegamiento (log2FoldChange).

Después se definen los Conjuntos de Genes: Se seleccionan conjuntos predefinidos de genes que representan vías biológicas, conjuntos de funciones o términos biológicos. Estos conjuntos a menudo se obtienen de bases de datos como Gene Ontology (GO), KEGG (Kyoto Encyclopedia of Genes and Genomes), Reactome, entre otras.

Luego se calula el Estadístico de Enriquecimiento: Este estadístico evalúa si los genes del conjunto están distribuidos de manera uniforme a lo largo de la lista ordenada o si están agrupados en una región específica.

Finalmente se evalúa la Significancia: Se evalúa la significancia estadística del enriquecimiento de cada conjunto de genes mediante pruebas de permutación o métodos similares.

Para la visualización de los resultados del análisis GSEA se pueden utilizar distintos gráficos. En nuestro caso será un bubble plot

In [ ]:
## Based on the scripts by Eva Sacristán and Sandra
#González (GENGS CBMSO)

suppressPackageStartupMessages({
  library(rstudioapi)
  library(DESeq2)
  library(clusterProfiler, quietly = TRUE)
  library(msigdbr, quietly = T)
  library(UpSetR, quietly = TRUE)
  library(enrichplot, quietly = TRUE)
  library(ggplot2)
})

Primero se cargan las librerías necesarias para los posteriores análisis.

In [ ]:
#Parameters
#Which database inside msigdbr?
category <- 'C2'
subcategory <- 'KEGG'
#msigdbr_collections()

#Will you use stat parameter for ordering or the shrinked log2fold change?
#Log2fold is traditional statistic, but lately t statistic has been more recommended
#https://www.biorxiv.org/content/10.1101/060012v3.full.pdf
statP <- T
#Plot the x top categories
topCat <- 15
#parameter for group comparison: 'males_', 'females_' or NULL
sexo <- 'females_'



Este script establece algunos parámetros para realizar un análisis de enriquecimiento de conjuntos de genes (GSEA) utilizando la base de datos MSigDB

1.   **Configuración de la Base de Datos MSigDB:**

  **category <- 'C2'** establece la categoría de conjuntos de genes en MSigDB. En este caso, se ha seleccionado la categoría 'C2', que contiene conjuntos de genes curados de bases de datos biológicas, como vías metabólicas, señalización celular, etc. Este parámetro lo modificaremos para conseguir toda la información que queramos de distintas bases de datos.

  **subcategory <- 'KEGG'** establece la subcategoría dentro de la categoría 'C2'. En este caso, se ha seleccionado 'KEGG', lo que indica que se utilizarán conjuntos de genes de la base de datos KEGG en MSigDB. Este parámetro también lo modificaremos

2.   **Parámetros para el Análisis:**

  **statP <- T** determina si se utilizará el valor estadístico (como t-statistic) o el logaritmo plegamiento ajustado (shrinked log2fold change) para ordenar los genes en la lista. En este caso, T indica que se utilizará el valor estadístico.

  **topCat <- 15** especifica cuántas de las categorías principales se visualizarán en el resultado del análisis. En este caso, se mostrarán las 15 categorías principales.

  **sexo <- 'females_'** Introduce un parámetro para la comparación entre grupos. En este caso, se utiliza 'females_' como un parámetro que podría indicar una comparación específica relacionada con género o sexo (por ejemplo, diferencias en conjuntos de genes entre grupos de hembras).

In [ ]:
#Paths
workingD <- rstudioapi::getActiveDocumentContext()$path
setwd(dirname(workingD))
#Inputte
input <- paste0('DEG_results_', sexo,'sinFamilia/deseq_objects.RData')

#Outputs
resD0 <-paste0('results_GSEA_', sexo,'sinFamilia/')
if (statP){
  resD1 <- paste0(resD0,'stat/')
} else {
  resD1 <- paste0(resD0, 'log2fold/')
}
resD <- gsub(':','_',paste0(resD1,category,'_', subcategory, '/'))
if (!file.exists(resD)){
  dir.create(file.path(resD), recursive = TRUE)
}

resTSV <- paste0(resD,'GSEA_results_', sexo,'sinFamilia.txt')
dotplotF <- paste0(resD, "dotplot_", sexo,"sinFamilia.jpeg")
geneconceptF <- paste0(resD,'gene_concept_net_', sexo,'sinFamilia.jpeg')
ridgeF <- paste0(resD,'GSEA_ridge_', sexo,'sinFamilia.jpeg')
upsetF <- paste0(resD,'upset_plot_', sexo,'sinFamilia.jpeg')
gseaplotsF <- paste0(resD,'all_gseaplots', sexo,'sinFamilia.jpeg')

Este script está estableciendo rutas de archivos y directorios para el análisis GSEA, específicamente para el procesamiento y almacenamiento de los resultados.

1. **Configuración del Directorio de Trabajo:**

  **workingD <- rstudioapi::getActiveDocumentContext()$path** obtiene la ruta del directorio de trabajo actual del documento activo en RStudio.

  **setwd(dirname(workingD))** establece el directorio de trabajo actual al directorio padre de la ubicación del documento activo. Esto se hace para asegurarse de que las rutas de archivos y directorios se definan correctamente.

2. **Definición de Rutas y Nombres de Archivos:**

  **input <- paste0('DEG_results_', sexo,'sinFamilia/deseq_objects.RData')** define la ruta al archivo que contiene los resultados del análisis de expresión génica diferencial.

3. **Definición de directorios para resultados:**

  **resD0 <-paste0('results_GSEA_', sexo,'sinFamilia/')** directorio principal para los resultados de GSEA.

  **resD1 <- paste0(resD0,'stat/') o resD1 <- paste0(resD0, 'log2fold/')** directorio específico dependiendo de si se utiliza el valor estadístico o el logaritmo del plegamiento para ordenar los genes.

  **resD <- gsub(':','_',paste0(resD1,category,'_', subcategory, '/'))** define el directorio de resultados específicos para la categoría y subcategoría seleccionadas, reemplazando los dos puntos (":") en el nombre de la categoría/subcategoría con guiones bajos.

  **if (!file.exists(resD)){dir.create(file.path(resD), recursive = TRUE)}** crea el directorio resD si no existe.

4. **Definición de nombres de archivos de salida:**

  **resTSV <- paste0(resD,'GSEA_results_', sexo,'sinFamilia.txt')**

  **dotplotF <- paste0(resD, "dotplot_", sexo,"sinFamilia.jpeg")**

  **geneconceptF <- paste0(resD,'gene_concept_net_', sexo,'sinFamilia.jpeg')**

  **ridgeF <- paste0(resD,'GSEA_ridge_', sexo,'sinFamilia.jpeg')**

  **upsetF <- paste0(resD,'upset_plot_', sexo,'sinFamilia.jpeg')**

  **gseaplotsF <- paste0(resD,'all_gseaplots', sexo,'sinFamilia.jpeg')**

In [ ]:
#1) Load data
load(input)

if(statP){
  res <- results(dds, contrast = c('Grupo', 'DH', 'C'))
  res <- res[complete.cases(res),]
  dat <- res$stat
  names(dat) <- as.character(rownames(res))
  dat <- sort(dat, decreasing=TRUE)
} else { #when using log2fold change it is necessary to shrink the values
  shrink <- lfcShrink(dds, coef = 12, type="apeglm", quiet =T)
  dat <- shrink$log2FoldChange
  names(dat) <- as.character(rownames(shrink))
  dat <- sort(dat, decreasing=TRUE)
}

Este bloque de código carga los resultados del análisis de expresión génica diferencial y prepara un conjunto de datos (dat) para su posterior uso en el análisis de enriquecimiento de conjuntos de genes (GSEA). La elección entre usar estadísticas o el logaritmo del plegamiento ajustado depende del valor de statP.

1. **Carga de Datos**:

  **load(input)** carga los resultados del análisis de expresión génica diferencial almacenados en el archivo especificado por la variable input. Este archivo generalmente contiene un objeto RData generado previamente que contiene los resultados del análisis de expresión génica diferencial.

2. **Selección de Estadística a Utilizar**:

  **if(statP) { ... } else { ... }** realiza una bifurcación basada en el valor de statP. Si statP es TRUE, utiliza la estadística (por ejemplo, t-statistic) para ordenar los genes; si es FALSE, utiliza el logaritmo del plegamiento ajustado (shrinked log2fold change).

    a. **Cuando statP es TRUE**:

      **res <- results(dds, contrast = c('Grupo', 'DH', 'C'))** extrae los resultados del análisis de expresión génica diferencial para la comparación de grupos especificada ('DH' frente a 'C').

      **res <- res[complete.cases(res),]** elimina filas con valores NA.

      **dat <- res$stat** selecciona la columna de estadísticas (puede ser t-statistic) de los resultados.

      **names(dat) <- as.character(rownames(res))** asigna nombres a los datos basados en los nombres de las filas de los resultados.

      **dat <- sort(dat, decreasing=TRUE)** ordena los datos de forma descendente.

    b. **Cuando statP es FALSE**:

      **shrink <- lfcShrink(dds, coef = 12, type="apeglm", quiet = T)** realiza el proceso de "shrinkage" (reducción) del logaritmo del plegamiento ajustado utilizando el método 'apeglm'.

      **dat <- shrink$log2FoldChange** selecciona el logaritmo del plegamiento ajustado.

      **names(dat) <- as.character(rownames(shrink))** asigna nombres a los datos basados en los nombres de las filas de los resultados "shrinked".

      **dat <- sort(dat, decreasing=TRUE)** ordena los datos de forma descendente.

In [ ]:
#2) Calculate GSEA and write tables of results
#Get genes and categories
db_sets <- msigdbr(species = 'Rattus norvegicus', category = category,
                   subcategory = subcategory)%>%
  dplyr::select(gs_name, ensembl_gene)
head(db_sets) #each gene associated with each msig group

Este código obtiene conjuntos de genes específicos de MSigDB para la especie 'Rattus norvegicus' y la categoría/subcategoría proporcionadas. La salida, almacenada en la variable db_sets, contendrá información sobre los genes asociados a cada conjunto de genes en la base de datos MSigDB.

**db_sets <- msigdbr(species = 'Rattus norvegicus', category = category, subcategory = subcategory)** utiliza la función msigdbr para obtener conjuntos de genes de MSigDB específicos para la especie 'Rattus norvegicus' y la categoría y subcategoría especificadas. Estos conjuntos de genes pueden representar vías biológicas, funciones celulares u otros conjuntos temáticos.

**dplyr::select(gs_name, ensembl_gene)** selecciona las columnas 'gs_name' (nombre del conjunto de genes) y 'ensembl_gene' (identificación ENSEMBL del gen) del objeto resultante.

**head(db_sets)** muestra las primeras filas del conjunto de genes obtenido, proporcionando una vista previa de los datos.

In [ ]:
#3) Perform GSEA
set.seed(1)
egs <- GSEA(geneList = dat, pvalueCutoff = 0.05, eps = 0, pAdjustMethod = "BH",
            seed = T, TERM2GENE = db_sets) #for more accurate p value set eps to 0
#https://bioconductor.org/packages/release/bioc/vignettes/fgsea/inst/doc/fgsea-tutorial.html
#head(egs@result)
egs_df <- data.frame(egs@result)
egs_df <- egs_df[, -c(1,2)]

write.table(egs_df, file = resTSV, sep= "\t", quote = F, row.names = T)

#Reconsider the top category number if there are less terms than especified
if (dim(egs_df)[1] < topCat){
  topCat <- dim(egs_df)[1]
}


Este bloque de código ejecuta el análisis GSEA, procesa los resultados y escribe un archivo de texto con información sobre los conjuntos de genes enriquecidos. También ajusta el número de categorías principales a mostrar (topCat) según el número real de categorías enriquecidas.

1. **Ejecución del Análisis GSEA**:

  **set.seed(1)** fija la semilla del generador de números aleatorios para asegurar reproducibilidad.

  **egs <- GSEA(geneList = dat, pvalueCutoff = 0.05, eps = 0, pAdjustMethod = "BH", seed = T, TERM2GENE = db_sets)** realiza el análisis GSEA utilizando la función GSEA del paquete fgsea. **geneList = dat** es la lista ordenada de genes para el análisis GSEA. **pvalueCutoff = 0.05** estbalece el umbral de p-value para considerar significativamente enriquecido un conjunto de genes.
  **eps = 0** establece epsilon a cero para obtener valores p más precisos. **pAdjustMethod = "BH"** ajusta el p-values utilizando el método de Benjamini-Hochberg. **seed = T** utiliza la semilla fijada anteriormente.**TERM2GENE = db_sets** son conjuntos de genes de MSigDB utilizados para el análisis.

2. **Procesamiento y Escritura de Resultados**:

  **egs_df <- data.frame(egs@result)** convierte los resultados del análisis GSEA a un marco de datos para facilitar su manipulación.

  **egs_df <- egs_df[, -c(1,2)]** elimina las columnas innecesarias del marco de datos resultante.

  **write.table(egs_df, file = resTSV, sep= "\t", quote = F, row.names = T)** escribe los resultados del análisis GSEA en un archivo de texto separado por pestañas (tsv). Esto incluye información sobre los conjuntos de genes enriquecidos y sus respectivos p-values ajustados.

3. **Ajuste del Número de Categorías Principales a Mostrar**:

  **if (dim(egs_df)[1] < topCat){topCat <- dim(egs_df)[1]}** verifica si hay menos categorías enriquecidas que el número especificado (topCat). Si es así, ajusta topCat para que sea igual al número real de categorías enriquecidas.

##Plots

###Dot plot

In [ ]:
#4) Plot the results
##Dotplot
jpeg(file = dotplotF, units = 'in', width = 15, height = 10,
     res = 300)
par(mar = c(2, 2, 2, 5))
title <- 'Dot plot with GSEA categories'
dotplot(egs, x = "GeneRatio", color = "p.adjust", showCategory = 15,
            font.size = 10, title = title)
invisible(dev.off())

Eeste bloque de código crea un dotplot para visualizar gráficamente las categorías enriquecidas obtenidas del análisis GSEA. Cada punto en el dotplot representa una categoría, y su posición vertical indica la proporción de genes enriquecidos en esa categoría. Los puntos están coloreados según su p-value ajustado.

**jpeg(file = dotplotF, units = 'in', width = 15, height = 10, res = 300)** inicia la creación de un archivo JPEG para almacenar el gráfico. Establece el tamaño y la resolución del gráfico.

**par(mar = c(2, 2, 2, 5))** ajusta los márgenes del gráfico para dejar espacio suficiente para etiquetas y títulos.

**title <- 'Dot plot with GSEA categories'** establece el título del gráfico.

**dotplot(egs, x = "GeneRatio", color = "p.adjust", showCategory = 15, font.size = 10, title = title)** crea el dotplot utilizando la función dotplot del paquete fgsea. **egs** objeto resultante del análisis GSEA. **x = "GeneRatio"**,utiliza la proporción de genes enriquecidos para ordenar el dotplot. **color = "p.adjust"** colorea los puntos según el p-value ajustado. **showCategory = 15** muestra las 15 categorías principales en el dotplot. **font.size = 10** establece el tamaño de la fuente en el dotplot. **title = title** utiliza el título especificado.

###Network plot

In [ ]:
##Gene-concept network
jpeg(file = geneconceptF, units = 'in', width = 15, height = 10, res = 300)
par(mar = c(2, 2, 2, 5))
cnetplot(egs, categorySize="p.adjust", font.size = 15, colorEdge = T)
invisible(dev.off())

Este script crea un gráfico de red que representa visualmente la asociación entre genes y conjuntos de genes basándose en los resultados del análisis GSEA. Los nodos del gráfico pueden representar genes o conjuntos de genes, y el tamaño de los nodos y el color de los bordes pueden indicar la relevancia estadística de la asociación entre genes y conceptos enriquecidos. Este tipo de visualización facilita la identificación de genes clave y sus conexiones con funciones biológicas o categorías específicas.

**jpeg(file = geneconceptF, units = 'in', width = 15, height = 10, res = 300)** inicia la creación de un archivo JPEG para almacenar el gráfico de red. Establece el tamaño y la resolución del gráfico.

**par(mar = c(2, 2, 2, 5))** ajusta los márgenes del gráfico para dejar espacio suficiente para etiquetas y títulos.

**cnetplot(egs, categorySize="p.adjust", font.size = 15, colorEdge = T)** utiliza la función cnetplot del paquete fgsea para generar el gráfico de red. **egs** objeto resultante del análisis GSEA. **categorySize="p.adjust"** utiliza el valor ajustado (p.adjust) como tamaño de los nodos en la red. Este tamaño puede reflejar la significancia estadística de la asociación de un gen con un conjunto de genes. **font.size = 15** establece el tamaño de la fuente en el gráfico de red. **colorEdge = T** colorea los bordes (edges) de la red, lo que puede resaltar las relaciones entre genes y conjuntos de genes.

**invisible(dev.off())** cierra el dispositivo gráfico, finalizando la creación del archivo JPEG.








###Ridge line plot

In [ ]:
##Ridge line plot
jpeg(file = ridgeF, units = 'in', width = 15, height = 10, res = 300)
par(mar = c(2, 2, 2, 5))
ridgeplot(egs, fill="p.adjust", orderBy= 'NES', core_enrichment = T,
          showCategory = topCat)
invisible(dev.off())


Este script crea un gráfico de línea de crestas que representa visualmente el enriquecimiento de conjuntos de genes basándose en los resultados del análisis GSEA. Cada cresta en el gráfico representa un conjunto de genes, y la posición y el color de las crestas reflejan la fuerza y significancia del enriquecimiento del conjunto de genes en las condiciones experimentales. Este tipo de visualización facilita la identificación de conjuntos de genes relevantes y su posición en términos de enriquecimiento.

**jpeg(file = ridgeF, units = 'in', width = 15, height = 10, res = 300)** inicia la creación de un archivo JPEG para almacenar el gráfico de línea de crestas. Establece el tamaño y la resolución del gráfico.

**par(mar = c(2, 2, 2, 5))** ajusta los márgenes del gráfico para dejar espacio suficiente para etiquetas y títulos.

**ridgeplot(egs, fill="p.adjust", orderBy= 'NES', core_enrichment = T, showCategory = topCat)** utiliza la función ridgeplot del paquete ggridges para generar el gráfico de línea de crestas. **egs** objeto resultante del análisis GSEA. **fill="p.adjust"** colorea las crestas según el valor ajustado (p.adjust), indicando la significancia estadística. **orderBy= 'NES'** ordena las crestas en el gráfico según el Score de Enriquecimiento Normalizado (NES por sus siglas en inglés), que es una medida de la fuerza y dirección del enriquecimiento del conjunto de genes. **core_enrichment = T** muestra solo los conjuntos de genes principales enriquecidos. **showCategory = topCat** muestra el número especificado de categorías principales en el gráfico de línea de crestas.

**invisible(dev.off())** cierra el dispositivo gráfico, finalizando la creación del archivo JPEG.

###Upset plot

In [ ]:
##Upset plot (of the 10 first terms)
#Save the genes in each category in a list
genes_top <- as.data.frame(as.factor(head(egs@result$core_enrichment, topCat)))
list_top <- list()
for (i in 1:topCat) {
    list_top[[i]] <- unlist(strsplit(as.character(genes_top[i,1]),split="/"))
}

Extrae los genes asociados a las categorías principales del resultado del análisis GSEA y los almacena en una lista.

In [ ]:
#Store all unique gene IDs
uniq <- as.character(unique(names(dat)))
#Get top functions
func_top <- egs$Description[1:topCat]

Obtiene todos los identificadores únicos de genes y las descripciones de las categorías principales.

In [ ]:
#Make sparse matrix with 1 for every gene in each category
mat <- matrix(0L, nrow = length(uniq), ncol = length(func_top))
for (gene in 1:length(uniq)) {
  for (func in 1:length(func_top)) {
    gen <- uniq[gene]
    if (gen %in% list_top[[func]]) {
      mat[gene,func] =  1
    }}}

Crea una matriz dispersa que indica la presencia o ausencia de cada gen en las categorías principales. Un valor de 1 indica la presencia del gen en la categoría, mientras que 0 indica ausencia.

In [ ]:
#Make a data frame
mat_df <- as.data.frame(mat)
colnames(mat_df) <- func_top
row.names(mat_df) <- uniq

Convierte la matriz a un marco de datos (data frame) donde las filas representan genes y las columnas representan categorías. Los nombres de las filas y columnas se establecen usando los genes y las descripciones de las categorías.

In [ ]:
#Plot
jpeg(file = upsetF, units = 'in', width = 15, height = 10, res = 300)
upset(mat_df, nsets=10, order.by="freq", sets.bar.color="skyblue")
invisible(dev.off())


Crea un gráfico Upset que muestra la intersección de los conjuntos de genes en las 10 categorías principales. Cada barra representa una categoría, y las intersecciones muestran qué genes están presentes en más de una categoría.

In [ ]:
###Plot the first 5 more abundant terms or all if there are less hits

for (j in 1:topCat){
  plot <- gseaplot2(egs, geneSetID=j, title = egs$Description[j], base_size=40, color="red")
  desc <- gsub(" ", "_", egs$Description[j], fixed = TRUE)
  filename <- paste0(resD, desc, ".jpeg")
  ggsave(plot, file=filename, device = "jpeg", units= "in", height = 15, width = 20)
}

Itera sobre las primeras 5 categorías (o las categorías superiores definidas por topCat) y crea un gráfico GSEA individual para cada una. El título del gráfico se toma de la descripción de la categoría, y el gráfico se guarda como un archivo JPEG.

In [ ]:
#For all top categories at once:
gseap <- gseaplot2(egs, geneSetID = 1:topCat, pvalue_table = F)
ggsave(gseap, file=gseaplotsF, device = "jpeg", units= "in",
       height = 15, width = 20)


Crea un gráfico GSEA combinado que muestra la información para todas las categorías principales al mismo tiempo. El gráfico se guarda como un archivo JPEG.